In [ ]:
!nvidia-smi

Thu Nov 23 18:55:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 67.5 M

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 2.0 MB/s eta 0:00:00
Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
Found existing installation: accelerate 0.24.1
Uninstalling accelerate-0.24.1:
  Successfully uninstalled accelerate-0.24.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 38.3 MB/s eta 0:00:00
  Using cached accelerate-0.24.1-py3-none-any.whl (261 kB)


In [ ]:
from transformers import pipeline, set_seed, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# To check the device using to run.
device = "cuda" if torch.cuda.is_available() else "CPU"
device

'cuda'

In [ ]:
# getting models for tokenizer and model
model_cpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_cpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_cpt)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
# Download and unzip
!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

--2023-11-23 18:57:26--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2023-11-23 18:57:27--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.02s   

2023-11-23 18:57:28 (313 MB/s) - ‘summarizer-data.zip’ saved [7903594/

In [ ]:
# Load Data
data_samsum  = load_from_disk("samsum_dataset")
data_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
# Data Analysis
split_length = [len(data_samsum[split]) for split in data_samsum]

print(f"length of data: {split_length}")
print(f"Features: {data_samsum['train'].column_names}")

print("\nDialogs")
print(data_samsum['train'][1]["dialogue"])

print("\nSummery")
print(data_samsum["train"][1]["summary"])

length of data: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogs
Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great

Summery
Olivia and Olivier are voting for liberals in this election. 


In [ ]:
# Tokenizing the data
def convert_data_to_tokens(example_batch):
  input_encoder = tokenizer(example_batch["dialogue"], max_length=1024, truncation=True)

  with tokenizer.as_target_tokenizer():
    target_encoder = tokenizer(example_batch["summary"], max_length=128, truncation=True)


  return {
      "input_ids": input_encoder["input_ids"],
      "attention_mask":input_encoder["attention_mask"],
      "labels":target_encoder["input_ids"]
  }

In [ ]:
# Tokenizing the data by passing it in batches
data_samsum_pt = data_samsum.map(convert_data_to_tokens, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
data_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [ ]:
# Training
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer

seq_2_seq_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

train_arg = TrainingArguments(
    output_dir="pegasus_semsum", num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay = 0.01, logging_steps=10,
    evaluation_strategy= 'steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

trainer = Trainer(
    model = model_pegasus, args = train_arg,
    tokenizer = tokenizer, data_collator = seq_2_seq_collator,
    train_dataset= data_samsum_pt["test"],
    eval_dataset= data_samsum_pt["validation"]
)
trainer.train()

You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=3.0754146295435287, metrics={'train_runtime': 161.1632, 'train_samples_per_second': 5.082, 'train_steps_per_second': 0.316, 'total_flos': 313317832187904.0, 'train_loss': 3.0754146295435287, 'epoch': 1.0})

In [ ]:
# Evaluation
def generate_batch_size_chunk(list_of_elements,batch_size):
  for i in range(0,len(list_of_elements),batch_size):
    yield list_of_elements[i : i+batch_size]

def calculate_metrics_on_test(dataset,metric,model,tokenizer,batch_size=16,device=device,column_text="article",column_summary="highlight"):
  article_batch = list(generate_batch_size_chunk(dataset[column_text], batch_size))
  target_batch = list(generate_batch_size_chunk(dataset[column_summary], batch_size))

  for article_batch, target_batch in tqdm(zip(article_batch,target_batch), total=len(article_batch)):
    input = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
    summaries = model.generate(input_ids=input["input_ids"].to(device),
                              attention_mask = input["attention_mask"].to(device),
                              length_penalty=0.8, num_beams=8, max_length=128)
    decode_summary = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
    decoded_summary = [s. replace(""," ") for s in decode_summary]

    metric.add_batch(predictions=decoded_summary, references=target_batch)

  score = metric.compute()
  return score


In [ ]:
rouge_names = ["rouge1","rouge2","rougeL","rougeLsum"]
rouge_metric = load_metric("rouge")

<ipython-input-15-93b142dbee1d>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


In [ ]:
#score = calculate_metrics_on_test(data_samsum["test"][0:10], rouge_metric, trainer.model, tokenizer, batch_size=2, column_text='dialogue', column_summary="summary")

score = calculate_metrics_on_test(
    data_samsum['test'][0:1], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rough_dist= dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rough_dist, index=[f'pegasus'])

100%|██████████| 5/5 [00:23<00:00,  4.72s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.017505,0.0,0.01502,0.014873


In [ ]:
# Prediction

## Save Model
model_pegasus.save_pretrained("Pegasus-samsum-model")

## Save Tokenizer
tokenizer.save_pretrained("tokenizer")

# load
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

# Prediction
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

sample_text = data_samsum["test"][0]["dialogue"]
reference = data_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model = model_pegasus, tokenizer=tokenizer)

print("\nDialogue")
print(sample_text)

print("\nReference")
print(reference)

print("\nPredicted")
print(pipe(sample_text,**gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)



Dialogue
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Predicted


RuntimeError: ignored